In [ ]:
import osiris_utils as ou
import numpy as np
from pathlib import Path
from tqdm import tqdm
import h5py
import sys
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


In [ ]:
sims = []
labels = []

#sim_Boris_0_01 = ou.Simulation("/home/exxxx5/Tese/Decks/ComparePushers/B_10_E_scaled_down/Boris/Boris_0_01/Boris_0_01.in")
#sims.append(sim_Boris_0_01)
#labels.append(r"$2\pi dt w_{ce} = 0.01$")

# sim_Boris_0_1 = ou.Simulation("/home/exxxx5/Tese/Decks/ComparePushers/B_10_E_scaled_down/Boris/Boris_0_1/Boris_0_1.in")
# sims.append(sim_Boris_0_1)
# labels.append(r"$2\pi dt w_{ce} = 0.1$")

sim_Boris_1 = ou.Simulation("/home/exxxx5/Tese/Decks/ComparePushers/B_10_E_scaled_down/Boris/Boris_1/Boris_1.in")
sims.append(sim_Boris_1)
labels.append(r"$2\pi dt w_{ce} = 1$")

sim_Boris_10 = ou.Simulation("/home/exxxx5/Tese/Decks/ComparePushers/B_10_E_scaled_down/Boris/Boris_10/Boris_10.in")
sims.append(sim_Boris_10)
labels.append(r"$2\pi dt w_{ce} = 10$")


In [ ]:
def get_trajectory(sim, particle, tmin = 0, tmax = 100000000000, unload=False):
    track = sim["test_electrons"]["tracks"]
    track.load_all()

    mask = (track.data["t"][particle, :] >= tmin) & (track.data["t"][particle, :] <= tmax)
    traj = np.stack([
        track.data["x1"][particle, :][mask],
        track.data["x2"][particle, :][mask],
        track.data["x3"][particle, :][mask],
        track.data["t"][particle, :][mask]
        ], axis=0)

    if unload:
        track.unload()

    return traj

def plot_trajectory(sim, label, particle, tmin = 0, tmax = 100000000000, unload=False, fig=None, ax=None):

    traj = get_trajectory(sim, particle, tmin, tmax, unload)
    x, y, z, t = traj[0, :], traj[1, :], traj[2, :], traj[3, :]
    
    if fig is None:
        fig = plt.figure(figsize=(10, 7))

    label1 = None
    label2 = None
    if ax is None:
        ax = fig.add_subplot(111, projection='3d')
        label1 = ("$t = {:.3f}$".format(t[0]) + "$[{}]$".format(sim["test_electrons"]["tracks"].units["t"]))
        label2 = ("$t = {:.3f}$".format(t[-1]) + "$[{}]$".format(sim["test_electrons"]["tracks"].units["t"]))


    if label == "Baseline":
        color = "black"
        markersize = 0
    else:
        color = None
        markersize = 1.5
    
    # Highlight start and end points
    ax.scatter(x[0], y[0], z[0], color='lightgreen', s=20, label=label1)
    ax.scatter(x[-1], y[-1], z[-1], color='r', s=20, label=label2)

    # Plot the trajectory
    ax.plot(x, y, z, marker='o', linestyle='-', markersize=markersize, linewidth=0.8, label=label, color=color)
    
    # Labels and title
    ax.set_xlabel(r'$x_1$')
    ax.set_ylabel(r'$x_2$')
    ax.set_zlabel(r'$x_3$')
    ax.legend()

    return fig, ax

In [ ]:
# Boris pusher trajectories
particle = 700
tmax=200


fig, ax = None, None                            
# fig, ax = plot_trajectory(sim_Boris_0_1, r"$2\pi dt w_{ce} = 0.1$", particle=particle, tmax=tmax, fig=fig, ax=ax)
fig, ax = plot_trajectory(sim_Boris_1, r"$2\pi dt w_{ce} = 1$", particle=particle, tmax=tmax, fig=fig, ax=ax)
fig, ax = plot_trajectory(sim_Boris_10, r"$2\pi dt w_{ce} = 10$", particle=particle, tmax=tmax, fig=fig, ax=ax)

ax.set_title("Boris pusher, particle " + str(particle))
plt.tight_layout()
plt.show()

